In [28]:
pip install gitpython

In [44]:
from git import Repo

repo_url = 'https://github.com/Natalya7oct/multilabel_classification.git'

clone_dir = 'mc4'

Repo.clone_from(repo_url, clone_dir)

<git.repo.base.Repo '/content/mc4/.git'>

In [30]:
!pip install transformers jsonlines

In [31]:
import warnings

warnings.filterwarnings('ignore')

In [32]:
import json
import jsonlines
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import functools
from collections import Counter
from argparse import Namespace
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm
import re
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import shutil
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import requests

In [45]:
import mc4.train_functions as tf
import mc4.get_datasets as gd

In [34]:
image_encoders_dict = {
    'resnet18': 512,
    'resnet50': 2048,
    'resnet152': 2048
}

In [35]:
text_encoders_dict = {
    'prajjwal1/bert-mini': 128,
    'bert-base-uncased': 768,
    'bert-large-uncased': 1024
}

In [36]:
args = Namespace()

args.savedir_multimodal = os.path.join('model_save', 'multimodal')
args.savedir_text = os.path.join('model_save', 'text')
args.savedir_image = os.path.join('model_save', 'image')
args.data_path_mmimdb= 'mmimdb'
args.data_path_coco= 'coco'

args.train_perc = 0.8
args.val_perc = 0.1
args.max_seq_len = 512
args.batch_sz = 4
args.n_workers = 2
args.img_embed_pool_type = 'avg'
args.num_image_embeds = 1
args.lr = 1e-5
args.lr_patience = 2
args.lr_factor = 0.5
args.max_epochs = 5
args.gradient_accumulation_steps = 16
args.patience = 10

In [37]:
os.makedirs(args.savedir_multimodal, exist_ok=True)
os.makedirs(args.savedir_text, exist_ok=True)
os.makedirs(args.savedir_image, exist_ok=True)
os.makedirs(args.data_path_mmimdb, exist_ok=True)
os.makedirs(f'{args.data_path_mmimdb}/dataset', exist_ok=True)
os.makedirs(args.data_path_coco, exist_ok=True)
os.makedirs(f'{args.data_path_coco}/dataset', exist_ok=True)

In [38]:
captions_url = 'https://drive.usercontent.google.com/u/0/uc?id=1x_Tr6C52Nt3Oq7ZBYsBnivwUrq-PhzaD&export=download'
instances_url = 'https://drive.usercontent.google.com/u/0/uc?id=1_tW8EpxC1ffyjWy3hruZGJZXUlcxE-Me&export=download'

'''
captions_url = 'https://drive.usercontent.google.com/u/0/uc?id=1onVMlzXqYmgmf-xp6jcu1zm5RVBR6UKp&export=download'
instances_url = 'https://drive.usercontent.google.com/u/0/uc?id=1p-AiOlPMIiE0mg7u6lLMjtpQTq1PWdtc&export=download'
'''

response = requests.get(captions_url)
captions = response.json()

response = requests.get(instances_url)
instances = response.json()

gd.write_formated_data_coco(args, captions, instances)

  0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
dataset_url = 'https://giohmg.am.files.1drv.com/y4mVMnd-_Cq2OqbN2YyYKBclz_PTvLBhmP1WICms-Dpmtj_gTCln0tbh-28SxECL1LE34uNcBRo2wu_7bLHaaBBqN1P990Nlw47RH-Loq8S2Sz8XpmPTWTuQfDJ0Gj5ZNUeDkq_cRLqTqHQCx51V-VHKJ1jHz1gDBFCCsB0xMgAPdRFJgRDR6CnjAyJjRighF7l_FnbcgHjAypnxwfsJomGYA'

'''
dataset_url = 'https://archive.org/download/mmimdb/mmimdb.tar.gz'
'''

response = requests.get(dataset_url)
if response.status_code == 200:
    with open('mmimdb.tar.gz', 'wb') as file:
        file.write(response.content)

!tar -zxf 'mmimdb.tar.gz'

gd.write_formated_data_mmimdb(args)

In [47]:
args.bert_type = 'bert-base-uncased' # 'prajjwal1/bert-mini' / 'bert-base-uncased'
args.text_hidden_sz = text_encoders_dict[args.bert_type]

args.resnet_type = 'resnet18' # 'resnet50' / 'resnet152'
args.img_hidden_sz = image_encoders_dict[args.resnet_type]

args.linear_layer_dim = 10
args.linear_layer_count = 0

model_type, params_count, test_f1 = tf.main(args, args.data_path_mmimdb)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Multimodel model


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 27.06 MiB is free. Process 2940 has 14.72 GiB memory in use. Of the allocated memory 13.97 GiB is allocated by PyTorch, and 616.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)